<a href="https://colab.research.google.com/github/tanvir-talha058/Machine-Learning/blob/main/hacking_news_with_mbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df = pd.read_excel('/content/hacker data.xlsx')
display(df.head())

,Title,Label
0,Critical Cosmos Database Flaw Affected Thousan...,Data_Breaches
1,Researchers Detail Modus Operandi of ShinyHunt...,Data_Breaches
2,Chinese Hackers Believed to be Behind Second C...,Data_Breaches
3,Using Breached Password Detection Services to ...,Data_Breaches
4,Can Data Protection Systems Prevent Data At Re...,Data_Breaches


# Task
Classify hacking news from the dataset "cybersecurity_news.csv" using the mbert model, applying 10 epochs for training, after preprocessing the data.

## Data preprocessing

### Subtask:
Preprocess the text data in the 'Title' column, which includes tokenization, handling special characters, and converting text to a suitable format for the mbert model.


**Reasoning**:
Preprocess the text data in the 'Title' column by cleaning, tokenizing, removing stop words, and joining the tokens back into strings.



In [2]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

df['Processed_Title'] = df['Title'].apply(preprocess_text)
display(df.head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


**Reasoning**:
The previous code failed because the 'punkt_tab' resource was not found. I will download 'punkt_tab' and re-run the preprocessing code.



In [3]:
nltk.download('punkt_tab')

def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

df['Processed_Title'] = df['Title'].apply(preprocess_text)
display(df.head())

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


,Title,Label,Processed_Title
0,Critical Cosmos Database Flaw Affected Thousan...,Data_Breaches,critical cosmos database flaw affected thousan...
1,Researchers Detail Modus Operandi of ShinyHunt...,Data_Breaches,researchers detail modus operandi shinyhunters...
2,Chinese Hackers Believed to be Behind Second C...,Data_Breaches,chinese hackers believed behind second cyberat...
3,Using Breached Password Detection Services to ...,Data_Breaches,using breached password detection services pre...
4,Can Data Protection Systems Prevent Data At Re...,Data_Breaches,data protection systems prevent data rest leakage


## Split data

### Subtask:
Split the data into training and testing sets.


**Reasoning**:
Split the data into training and testing sets using train_test_split.



In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Processed_Title'], df['Label'], test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (2993,)
X_test shape: (749,)
y_train shape: (2993,)
y_test shape: (749,)


## Load mbert model

### Subtask:
Load the pre-trained mbert model and tokenizer from the `transformers` library.


**Reasoning**:
Load the pre-trained mbert model and tokenizer from the `transformers` library.



In [5]:
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(df['Label'].unique()))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Fine-tune mbert

### Subtask:
Fine-tune the mbert model on your dataset for 10 epochs.


**Reasoning**:
Import the necessary classes for training and evaluation, define a function to tokenize the data, create a custom dataset class, instantiate the datasets, define training arguments, instantiate the Trainer, and start the training process.



In [6]:
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset

def tokenize_function(examples):
    return tokenizer(examples, padding="max_length", truncation=True)

class NewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_encodings = tokenize_function(list(X_train))
test_encodings = tokenize_function(list(X_test))

train_labels = y_train.astype('category').cat.codes.values
test_labels = y_test.astype('category').cat.codes.values

train_dataset = NewsDataset(train_encodings, train_labels)
test_dataset = NewsDataset(test_encodings, test_labels)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ahmed15-5307 (ahmed15-5307-daffodil-international-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


ValueError: Target size (torch.Size([8])) must be the same as input size (torch.Size([8, 4]))

**Reasoning**:
The traceback indicates a `ValueError` because the target size does not match the input size in the loss calculation. This is likely due to the model expecting multi-label classification output (logits with size [batch_size, num_labels]) but receiving single-label targets (labels with size [batch_size]). The model should be initialized for single-label classification. Additionally, the labels are not in the correct format (one-hot encoded) for the loss function. The labels need to be one-hot encoded to match the output shape of the model. The target size should match the input size of the logits from the model, which is [batch_size, num_labels]. The labels should be transformed into a tensor with this shape, where each row is a one-hot encoding of the class label. The error also shows that the model is initialized with `num_labels=4` which is correct for the 4 classes in the dataset, but the target size is `[8]` which indicates single label targets.



In [7]:
import torch

def tokenize_function(examples):
    return tokenizer(examples, padding="max_length", truncation=True)

class NewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

train_encodings = tokenize_function(list(X_train))
test_encodings = tokenize_function(list(X_test))

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_labels = le.fit_transform(y_train)
test_labels = le.transform(y_test)


train_dataset = NewsDataset(train_encodings, train_labels)
test_dataset = NewsDataset(test_encodings, test_labels)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    report_to="none" # Disable wandb reporting
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

ValueError: Target size (torch.Size([8])) must be the same as input size (torch.Size([8, 4]))

**Reasoning**:
The traceback still shows the same `ValueError`, indicating that the model is expecting multi-label classification targets (size [batch_size, num_labels]), but the labels are still being provided as single-label targets (size [batch_size]). This is because the model was initialized with `problem_type` implicitly set to "multi_label_classification" when `num_labels > 1`. To fix this, the model needs to be explicitly configured for single-label classification by setting `problem_type` to "single_label_classification" and using `CrossEntropyLoss` as the loss function. The labels are already correctly encoded as integers using `LabelEncoder`. The labels should also be converted to a torch tensor of type `long`.



In [13]:
from transformers import TrainingArguments, Trainer, BertForSequenceClassification
from torch.utils.data import Dataset
import torch
from sklearn.preprocessing import LabelEncoder

def tokenize_function(examples):
    return tokenizer(examples, padding="max_length", truncation=True)

class NewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

train_encodings = tokenize_function(list(X_train))
test_encodings = tokenize_function(list(X_test))

le = LabelEncoder()
train_labels = le.fit_transform(y_train)
test_labels = le.transform(y_test)

train_dataset = NewsDataset(train_encodings, train_labels)
test_dataset = NewsDataset(test_encodings, test_labels)

model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(le.classes_), problem_type="single_label_classification")

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    report_to=["all"] # Enable reporting to save logs
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


KeyboardInterrupt: 

## Evaluate model

### Subtask:
Evaluate the fine-tuned model on the test set to assess its performance.


**Reasoning**:
Use the trained model to evaluate its performance on the test dataset and print the results.



In [9]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 1.6377462148666382, 'eval_runtime': 21.8943, 'eval_samples_per_second': 34.21, 'eval_steps_per_second': 4.293, 'epoch': 10.0}


## Classify news

### Subtask:
Use the trained model to classify new hacking news articles.


**Reasoning**:
Create new news articles, tokenize them using the trained tokenizer, and prepare them for prediction.



In [10]:
import torch
from torch.utils.data import Dataset

# 1. Create a list or pandas Series containing the new hacking news articles
new_articles = [
    "Major cybersecurity firm reports a new ransomware variant targeting healthcare.",
    "Researchers discover a critical zero-day vulnerability in popular web browser.",
    "Phishing campaign impersonates government agencies to steal personal information.",
    "New report highlights the rise of AI-powered cyberattacks.",
    "Data breach exposes millions of customer records from an online retailer."
]

# 2. Tokenize these new articles using the same tokenizer that was used during training.
# Ensure padding and truncation settings are consistent.
new_encodings = tokenizer(new_articles, padding="max_length", truncation=True, return_tensors="pt")

# 3. Convert the tokenized input into a PyTorch dataset compatible with the trained model.
class PredictDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

predict_dataset = PredictDataset(new_encodings)

**Reasoning**:
Use the trained trainer to predict the labels for the new articles and then decode the predictions into meaningful class labels.



In [11]:
import numpy as np

# 4. Use the trainer.predict() method to get predictions for the new dataset.
predictions = trainer.predict(predict_dataset)

# Get the predicted logits
predicted_logits = predictions.predictions

# Get the predicted class indices by finding the index of the maximum logit for each sample
predicted_indices = np.argmax(predicted_logits, axis=1)

# 5. Convert the predicted logits into class labels using the inverse transform of the LabelEncoder
predicted_labels = le.inverse_transform(predicted_indices)

# 6. Display the original new articles along with their predicted labels.
for article, label in zip(new_articles, predicted_labels):
    print(f"Article: {article}\nPredicted Label: {label}\n")

/tmp/ipython-input-1591677429.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Article: Major cybersecurity firm reports a new ransomware variant targeting healthcare.
Predicted Label: Cyber_Attack

Article: Researchers discover a critical zero-day vulnerability in popular web browser.
Predicted Label: Vulnerability

Article: Phishing campaign impersonates government agencies to steal personal information.
Predicted Label: Malware

Article: New report highlights the rise of AI-powered cyberattacks.
Predicted Label: Cyber_Attack

Article: Data breach exposes millions of customer records from an online retailer.
Predicted Label: Data_Breaches



## Summary:

### Data Analysis Key Findings

*   The dataset was successfully preprocessed by cleaning the text in the 'Title' column, tokenizing it, and removing stop words. The processed text was stored in a new column, 'Processed\_Title'.
*   The dataset was split into training and testing sets with an 80/20 ratio, resulting in 2993 samples for training and 749 samples for testing.
*   The 'bert-base-multilingual-cased' model and its corresponding tokenizer were successfully loaded. The model was initialized for sequence classification with the correct number of output labels based on the unique labels in the dataset.
*   The mbert model was successfully fine-tuned on the processed dataset for 10 epochs. The training process was configured for single-label classification, and the labels were correctly integer-encoded.
*   The fine-tuned model achieved an evaluation loss of approximately 1.638 on the test set.
*   The trained model was successfully used to classify new hacking news articles, and the predicted labels were generated for each article.

### Insights or Next Steps

*   The evaluation loss of 1.638 suggests that there is room for improvement in the model's performance. Further optimization of hyperparameters, exploring different models, or increasing the training data size could potentially lead to better results.
*   To provide a more comprehensive evaluation, additional metrics such as accuracy, precision, recall, and F1-score should be calculated and analyzed on the test set.


In [12]:
import matplotlib.pyplot as plt
import seaborn as sns
import json

# Load the trainer logs
log_history_path = './results/trainer_state.json'
with open(log_history_path, 'r') as f:
    trainer_state = json.load(f)

log_history = trainer_state['log_history']

# Filter for training loss
train_loss_steps = []
train_loss_values = []
for log in log_history:
    if 'loss' in log:
        train_loss_steps.append(log['step'])
        train_loss_values.append(log['loss'])

# Plot training loss
plt.figure(figsize=(10, 6))
sns.lineplot(x=train_loss_steps, y=train_loss_values)
plt.xlabel("Training Steps")
plt.ylabel("Training Loss")
plt.title("Training Loss over Steps")
plt.grid(True)
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: './results/trainer_state.json'